In [1]:
import os
from datetime import datetime as dt
from random import randint
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')

import torchaudio
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from tensorboardX import SummaryWriter

from hparams import Hparam
from data.dataset import SpeechDataset
from CPC.model import CPCModel
# from CPC_classifiers.speaker_model import SpeakerClassificationCPC

In [31]:
config = Hparam('./CPC/config.yaml')
# config.train.device = 'cpu'
gettime = lambda: str(dt.time(dt.now()))[:8]

## model

In [32]:
ds = SpeechDataset(config.data.path)
train_ds, test_ds = train_test_split(ds, test_size=0.2)

100%|██████████| 2703/2703 [00:06<00:00, 426.78it/s]


In [33]:
model_cpc = CPCModel(config).to(config.train.device)
model_cpc.load_state_dict(torch.load('checkpoints/cpc_model_35_epoch.pt', map_location=config.train.device))

<All keys matched successfully>

## experiments

In [16]:
sample = ds[27][1].unsqueeze(0).unsqueeze(0).to(config.train.device)
sample

tensor([[[0.0014, 0.0014, 0.0002,  ..., 0.0060, 0.0050, 0.0033]]],
       device='cuda:0')

In [20]:
out = model_cpc.predict(sample)
[t.size() for t in out]

[torch.Size([1, 128, 256]), torch.Size([1, 1, 256])]

## workbench

In [19]:
c = nn.Conv1d(5, 5, kernel_size=1)

In [20]:
c.weight.size()

torch.Size([5, 5, 1])

In [21]:
t = torch.Tensor(1, 5, 166)

In [22]:
c(t).size()

torch.Size([1, 5, 166])

In [30]:
c.state_dict()

OrderedDict([('weight', tensor([[[ 0.3509],
                       [ 0.2020],
                       [-0.3632],
                       [ 0.0610],
                       [-0.0119]],
              
                      [[ 0.1810],
                       [-0.1683],
                       [ 0.3941],
                       [-0.1763],
                       [-0.1956]],
              
                      [[-0.3396],
                       [ 0.3310],
                       [-0.2803],
                       [ 0.3847],
                       [-0.1057]],
              
                      [[-0.0631],
                       [ 0.1442],
                       [-0.3527],
                       [ 0.4251],
                       [-0.0659]],
              
                      [[-0.4312],
                       [-0.0440],
                       [ 0.4193],
                       [-0.0884],
                       [-0.0117]]])),
             ('bias',
              tensor([4.4219e-01, 1.9525e-02, 5.41

In [34]:
model_cpc.get_summary(ds[1][1])

                                     Kernel Shape    Output Shape     Params  \
Layer                                                                          
0_convolutions.Conv1d_0              [1, 512, 10]  [1, 512, 4095]     5.632k   
1_convolutions.ReLU_1                           -  [1, 512, 4095]          -   
2_convolutions.BatchNorm1d_2                [512]  [1, 512, 4095]     1.024k   
3_convolutions.Conv1d_3             [512, 512, 8]  [1, 512, 1023]  2.097664M   
4_convolutions.ReLU_4                           -  [1, 512, 1023]          -   
5_convolutions.BatchNorm1d_5                [512]  [1, 512, 1023]     1.024k   
6_convolutions.Conv1d_6             [512, 512, 4]   [1, 512, 512]  1.049088M   
7_convolutions.ReLU_7                           -   [1, 512, 512]          -   
8_convolutions.BatchNorm1d_8                [512]   [1, 512, 512]     1.024k   
9_convolutions.Conv1d_9             [512, 512, 4]   [1, 512, 257]  1.049088M   
10_convolutions.ReLU_10                 

In [38]:
out = model_cpc.predict(ds[1][1].unsqueeze(0).unsqueeze(0).cuda())
[t.size() for t in out]

[torch.Size([1, 128, 256]), torch.Size([1, 1, 256])]